In [4]:
from datetime import datetime
import dask.array as da
import os
import time
import h5py
import numpy as np
import xarray as xr
import downscale as down
import matplotlib.pyplot as plt

import warnings

import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

In [5]:
# resolution = '3h'
resolution = 'daily'

In [8]:
tmpa_dir = os.path.join('..', 'data', 'tmpa_conus_data')
inputfile = os.path.join(tmpa_dir, f'data_tmpa_{resolution}.hdf5')
outfile = os.path.join(tmpa_dir, f'data_tmpa_{resolution}_cut.nc')

print(f'Input file : {inputfile}')
print(f'Output file: {outfile}')

Input file : ../data/tmpa_conus_data/data_tmpa_daily.hdf5
Output file: ../data/tmpa_conus_data/data_tmpa_daily_cut.nc


In [10]:
ncf

<xarray.Dataset> Size: 38MB
Dimensions:  (phony_dim_0: 305, phony_dim_1: 112, phony_dim_2: 280)
Dimensions without coordinates: phony_dim_0, phony_dim_1, phony_dim_2
Data variables:
    dates    (phony_dim_0) int32 1kB ...
    lat      (phony_dim_1) float32 448B ...
    lon      (phony_dim_2) float32 1kB ...
    prcp     (phony_dim_2, phony_dim_1, phony_dim_0) float32 38MB ...

In [29]:
ncf = xr.open_dataset(inputfile, engine="h5netcdf",phony_dims='sort')

tmpalat = ncf['lat'][:].values
tmpalon = ncf['lon'][:].values
dates_int = ncf['dates'][:].values
dset = ncf['prcp'].values

if resolution == '3h':
    time_int = ncf['hours'][:].values
    dates = [datetime.strptime(str(integd)+str(inthour), '%Y%m%d%H') for integd, inthour in zip(dates_int, time_int)]
    description = "TMPA 3h"
elif resolution == 'daily':
    time_int = ncf['dates'][:].values
    dates = ncf['dates'][:].values
    description = "TMPA daily"
else:
    print('ERROR resolution dont found it')

del ncf

In [25]:
min_lon = -96
max_lon = -90
min_lat = 30
max_lat = 36

In [27]:
x = da.from_array(dset, chunks=(6, 6, 300))

xrs0 = xr.DataArray(x,  coords={'lon':tmpalon, 'lat':tmpalat, 'time':dates}, dims=('lon', 'lat', 'time'))
xrs0 = xrs0.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))

In [28]:
pre = np.array(xrs0.data)
pre = np.swapaxes(pre,0,2)
pre.shape

(305, 24, 24)

In [30]:
PRE_xr = xr.Dataset(data_vars={"PRE": (("time", "lat", "lon"), pre)},
                    coords={'time': dates, 'lat': xrs0['lat'].values, 'lon': xrs0['lon'].values},
                    attrs=dict(description=description))
PRE_xr.PRE.attrs["units"] = "millimeter"
PRE_xr.lat.attrs["units"] = "degrees_north"
PRE_xr.lon.attrs["units"] = "degrees_east"

In [33]:
print(f'Export PRE data to {outfile}')
PRE_xr.to_netcdf(outfile)

Export PRE data to ../data/tmpa_conus_data/data_tmpa_daily_cut.nc


In [34]:
PRE_xr

<xarray.Dataset> Size: 704kB
Dimensions:  (time: 305, lat: 24, lon: 24)
Coordinates:
  * time     (time) int32 1kB 20080101 20180101 20180102 ... 20181030 20181031
  * lat      (lat) float32 96B 30.12 30.38 30.62 30.88 ... 35.38 35.62 35.88
  * lon      (lon) float32 96B -95.88 -95.62 -95.38 ... -90.62 -90.38 -90.12
Data variables:
    PRE      (time, lat, lon) float32 703kB -9.999e+03 -9.999e+03 ... 8.39 2.055
Attributes:
    description:  TMPA daily